In [ ]:
import numpy as np
import xarray as xr
import numpy.matlib
from fetutorialfunctions import adjustRhoQvInitial

In [ ]:
path_base = "INSERT_PATH_TO_YOUR_RUN_DIRECTORY"
file_FE_0 = path_base + '/output/FE_OFFSHORE.0'
file_FE_0_ini = path_base + '/initial/FE_OFFSHORE.0'

In [ ]:
fe_data = xr.open_dataset(file_FE_0)
fe_data

# Assign 1.0 to SeaMask required for offshore roughness parameterizations

In [ ]:
SeaMask = fe_data.SeaMask.isel(time=0).values

In [ ]:
SeaMask_n = np.zeros(SeaMask.shape)
SeaMask_n[:,:] = 1.0

# Create initial condition for qv and adjust rho accordinly 

In [ ]:
# From FastEddy's parameters file
BS_Dict = {'stabilityScheme': 2,
           'temp_grnd': 288.000,
           'pres_grnd': 100000.0,
           'zStableBottom': 925.0,
           'stableGradient': 0.003,
           'zStableBottom2': 1075.0,
           'stableGradient2': 0.003,
           'zStableBottom3': 50000.0,
           'stableGradient3': 0.003,}

In [ ]:
# User-specified qv profile
zq_lev = [0.0, 925.0, 1075.0, 5000.0] # m
qv_lev = [10.0, 8.15, 7.85, 0.0] # g/kg
qv_skin = 14.0 # g/kg
###
dqdz = np.zeros(len(zq_lev)-1)
dqdz[0] = (qv_lev[1]-qv_lev[0])/zq_lev[1]
dqdz[1] = (qv_lev[2]-qv_lev[1])/(zq_lev[2]-zq_lev[1])
dqdz[2] = (qv_lev[3]-qv_lev[2])/(zq_lev[3]-zq_lev[2])
print('dqdz=',dqdz)

In [ ]:
# Vertical grid (1d profile)
z_prof_1d = np.squeeze(fe_data.zPos.isel(time=0,yIndex=0,xIndex=0).values)

In [ ]:
rhom_prof,qv_prof = adjustRhoQvInitial(z_prof_1d, BS_Dict, qv_lev, zq_lev, dqdz)

In [ ]:
rhom_2d = np.tile(rhom_prof, [fe_data.sizes['xIndex'], 1])
print('rhom_2d.shape=',rhom_2d.shape)
rhom_3d = np.tile(rhom_2d, [fe_data.sizes['yIndex'], 1, 1])
print('rhom_3d.shape=',rhom_3d.shape)
rhom_3d_n = np.transpose(rhom_3d,[2,0,1])
print('rhom_3d_n.shape=',rhom_3d_n.shape)

In [ ]:
qv_2d = np.tile(qv_prof, [fe_data.sizes['xIndex'], 1])
print('rhom_2d.shape=',rhom_2d.shape)
qv_3d = np.tile(qv_2d, [fe_data.sizes['yIndex'], 1, 1])
print('qv_3d.shape=',qv_3d.shape)
qv_3d_n = np.transpose(qv_3d,[2,0,1])
print('qv_3d_n.shape=',qv_3d_n.shape)

In [ ]:
ds_out = fe_data

In [ ]:
ds_out['SeaMask'][0,:,:] = 1.0
ds_out['qskin'][0,:,:] = qv_skin
ds_out['qv'][0,:,:,:] = qv_3d_n
ds_out['rho'][0,:,:,:] = rhom_3d_n

In [ ]:
ds_out.to_netcdf(file_FE_0_ini)